In [1]:
%reload_ext autoreload
%autoreload 2

from core.experiments import RunFactory

In [2]:
wandb_user = 'sisaman'
wandb_project = 'GAP-TEST'

run_factory = RunFactory(entity=wandb_user, project=wandb_project, check_existing=True)

# Experiments

### Hyper-parameters

In [3]:
datasets = ['facebook', 'reddit', 'amazon']
batch_size = {'facebook': 256, 'reddit': 2048, 'amazon': 4096}

gap_methods  = ['gap-inf', 'gap-edp', 'gap-ndp']
sage_methods = ['sage-inf', 'sage-edp', 'sage-ndp']
mlp_methods  = ['mlp', 'mlp-dp']
inf_methods  = ['gap-inf', 'sage-inf']
edp_methods  = ['gap-edp', 'sage-edp', 'mlp']
ndp_methods  = ['gap-ndp', 'sage-ndp', 'mlp-dp']
all_methods  = inf_methods + edp_methods + ndp_methods

hparams = {dataset: {method: {} for method in all_methods} for dataset in datasets}

for dataset in datasets:
    # For GAP methods
    for method in gap_methods:
        hparams[dataset][method]['encoder_layers'] = 2
        hparams[dataset][method]['base_layers'] = 1
        hparams[dataset][method]['head_layers'] = 1
        hparams[dataset][method]['combine'] = 'cat'
        hparams[dataset][method]['hops'] = [1, 2, 3, 4, 5]

    # For SAGE methods
    for method in sage_methods:
        hparams[dataset][method]['base_layers'] = 2
        hparams[dataset][method]['head_layers'] = 1
        if method != 'sage-ndp':
            hparams[dataset][method]['mp_layers'] = [1, 2, 3, 4, 5]
        
    # For MLP methods
    for method in mlp_methods:
        hparams[dataset][method]['num_layers'] = 3

    # For GAP-NDP and SAGE-NDP
    for method in ['gap-ndp', 'sage-ndp']:
        hparams[dataset][method]['max_degree'] = [100, 200, 300, 400]

    # For all methods
    for method in all_methods:
        hparams[dataset][method]['hidden_dim'] = 16
        hparams[dataset][method]['activation'] = 'selu'
        hparams[dataset][method]['optimizer'] = 'adam'
        hparams[dataset][method]['learning_rate'] = 0.01
        hparams[dataset][method]['repeats'] = 10
        
        if method in ndp_methods:
            hparams[dataset][method]['max_grad_norm'] = 1
            hparams[dataset][method]['epochs'] = 10
            hparams[dataset][method]['batch_size'] = batch_size[dataset]
        else:
            hparams[dataset][method]['batch_norm'] = True
            hparams[dataset][method]['epochs'] = 100
            hparams[dataset][method]['batch_size'] = 'full'


    # For GAP methods
    for method in gap_methods:
        hparams[dataset][method]['encoder_epochs'] = hparams[dataset][method]['epochs']


### Accuracy/Privacy Trade-off

In [4]:
for dataset in datasets:
    for method in all_methods:
        params = {}
        if method in ndp_methods:
            params['epsilon'] = [1, 2, 4, 8, 16]
        elif method in ['gap-edp', 'sage-edp']:
            params['epsilon'] = [0.1, 0.2, 0.5, 1, 2, 4, 8]
            
        run_factory.register(
            method, 
            dataset=dataset,
            **params, 
            **hparams[dataset][method]
        )

print(len(run_factory.get_all_runs()))

618


### Effect of Encoder Module

In [5]:
for dataset in datasets:
    for method in ['gap-edp', 'gap-ndp']:
        hp = {**hparams[dataset][method]}
        default_encoder_layers = hp.pop('encoder_layers')
        epsilon = [0.5, 1, 2, 4, 8] if method == 'gap-edp' else [1, 2, 4, 8, 16]
        run_factory.register(
            method,
            dataset=dataset,
            encoder_layers=[0, default_encoder_layers],
            epsilon=epsilon,
            **hp
        )

print(len(run_factory.get_all_runs()))

993


### Effect of Hops

In [6]:
for dataset in datasets:
    for method in ['gap-edp', 'gap-ndp']:
        hp = {**hparams[dataset][method]}
        hp.pop('hops')
        hops = [1,2,3,4,5]
        epsilon = [1, 2, 4, 8] if method == 'gap-edp' else [2, 4, 8, 16]
        run_factory.register(
            method,
            dataset=dataset,
            hops=hops,
            epsilon=epsilon,
            **hp
        )

print(len(run_factory.get_all_runs()))

993


### Effect of Degree

In [7]:
for dataset in datasets:
    method = 'gap-ndp'
    hp = {**hparams[dataset][method]}
    hp.pop('max_degree')
    max_degree = [10,20,50,100,200,300,400]
    epsilon = [2, 4, 8, 16]
    run_factory.register(
        method,
        dataset=dataset,
        max_degree=max_degree,
        epsilon=epsilon,
        **hp
    )

print(len(run_factory.get_all_runs()))

1173


In [8]:
run_factory.save(path='jobs/gap.sh')

Saved 1173 runs to jobs/gap.sh
